## 힌국복지패널 데이터 분석
- 한국보건사회연구원에서 우리나가 가구의 경제활동을 연구한 데이터
- 전국 7,000여 가구를 선정하여 2006년 부터 매년 추적 조사한 자료
- 경제활동, 생활실태, 복지욕구 등 천여개 변수로 구성됨
- 통계 패키지 SPSS 전용 파일
- bit.ly/easypy_91

## 한국복지패널 데이터 분석 준비하기

### Lab 1 : 데이터 분석 준비하기

1. 데이터 준비하기
- Koweps_hpwc14_2019_beta2.sav vkdlf ekdnsfhem
- 202년 발간된 복지패널 데이터로, 6331가구, 14,418명 정보를 담고 있음

2. 패키지 설치 및 로드하기
- 실습에 사용항 데이터 파일은 통계 분석 패키지인 SPAA 전용 파일임
- pyreadstat 패키지를 설치하면 SPSS, SAS, STATA등 다양한 통계 분석 소프트웨어 데이터 파일을 불러 올 수 있음
- pip install pyreadstat

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
pip install pyreadstat

3. 데이터 불러오기
- pd.read_spss()로 데이터 불러오기
- df.copy()로 복사본을 만들어 분석에 활용함

In [ ]:
# 데이터 불러오기
raw_welfare = pd.read_spss('Koweps_hpwc14_2019_beta2.sav')

In [ ]:
# 복사본 만들기
welfare = raw_welfare.copy()

In [ ]:
welfare.head()

4. 데이터 검토하기
- 데이터의 특성을 살펴보기
- 행, 열 개수
- 변수 속성
- 요약 통계양

In [ ]:
welfare.head()

In [ ]:
welfare.tail()

In [ ]:
# 행과 열 개수 출력
welfare.shape

In [ ]:
# 변수 속성 출력
welfare.info()

In [ ]:
# 요약 통계량
welfare.describe()

5. 변수명 바꾸기
- 복지데이터와 같은 대규모 데이토는 변수의 수가 많고 변수명이 코드로 되어 있음
- 규모가 큰 조사 자료의 경우 데이터 특징을 설명한 코드북(codebook)을 함께 제공
- 코드북에는 변수명과 값의 의미가 설명되어 있음
- 복지 데이터 코드북 일부: Koweps_Codebook_2019.xlsl
- 7개 변수를 선정하여 분석함
   - (1단계) 변수검토 및 전처리
   - (2단계) 변수 간 관계 분석

In [ ]:
welfare = welfare.rename(
   columns = {'h14_g3'     : 'sex',             # 성별
              'h14_g4'     : 'birth',           # 생년
              'h14_g10'    : 'marriage_type',   # 결혼 유무
              'h14_g11'    : 'religion',        # 종교
              'p1402_8aq1' : 'income',          # 수입(연봉)
              'h14_eco9'   : 'code_job',        # 직업 분류
              'h14_reg7'   : 'code_region'      # 지역
             })

In [ ]:
# 성별로 몇명이 있는지 확인
# 성별 변수의 데이터 타입 확인
welfare['sex'].dtypes

In [ ]:
# 성별로 몇명이 있는지 확인
# 성별 분류별 빈도 수 구하기 (성별은 1, 2로 구성)
welfare['sex'].value_counts()

- 성별 값에 대한 코드북 확인
- 여자 7913명, 남자 6505명
- 모름/무응답(9) 값은 없는 것으로 확인됨
|   값     |   내용      |
| ---      | ----        |
| 1        | 남자        |
| 2        | 여자        |
| 9        | 모름/무응답 |

In [ ]:
# 성별 항목 이름 부여
welfare['sex'] = np.where(welfare['sex'] == 1, 'male', 'female')

- 연령대별 기준을 정하여 파생변수 생성
|    범주     |     기준     |
| ---         | -----        |
|초년층       | 30세 미만    |
|중년층       | 30세 ~ 59세  |
|노년층       | 60세 이상    |

In [ ]:
# 파생 변수 나이(age) 추가하기
welfare = welfare.assign(age = 2019 - welfare['birth'] + 1)

In [ ]:
# 파생변수 age의 요약 통계량 구하기
welfare['age'].describe()

In [ ]:
# 나이 변수 살펴 보기
welfare['age'].head()

In [ ]:
# 연령대 변수 만들기
welfare = welfare.assign(ageg = np.where(welfare['age'] < 30, 'young',
                                np.where(welfare['age'] <=59, 'middle',
                                                               'old' )))

In [ ]:
# 연령대별 빈도 구하기
welfare['ageg'].value_counts()

## 직업별 월급 차이 - 어떤 직업이 월급을 가장 많이 받을까?

### Lab 2 : 직업 변수 검토 및 전처리하기


1. 변수 검토하기

In [ ]:
# 직업 변수 타입 살펴보기
welfare['code_job'].dtypes

In [ ]:
# 직업 코드별 빈도수 구하기
welfare['code_job'].value_counts()

In [ ]:
welfare['code_job'].head(10)

> code_job은 직업 분류 코드를 의미한다.
- 한국복지패널 사이트에서 제공하는 코드북의 직업분류코드목록(한국표준직업분류 제7차 개정)을 보면 직업 의미를 알 수 있음

2. 전처리 하기
- Koweps_Codebook_2019.xlsx 파일의 직종 코드 시트에 직업분류코드 목록을 불러옴
- 코드값과 직업명으로 구성되고 156개 직업으로 분류됨

In [ ]:
#직업명 불러오기
list_job = pd.read_excel('Koweps_Codebook_2019.xlsx', sheet_name='직종코드')

In [ ]:
# 직업 코드별 직업명 알아보기
list_job.head()

In [ ]:
# 직업 코드별 직업명 데이터프레임 구조 (156개 행, 2개 열)
list_job.shape

In [ ]:
# welfare와 list_job 결합(직업코드에 해당하는 직업명 넣기)
welfare = welfare.merge(list_job, how='left', on='code_job')

In [ ]:
# welfare에 직업코드와 직업명 출력
welfare[['code_job', 'job']].head()

In [ ]:
# welfare에 직업코드에서 결측치 제거
welfare.dropna(subset=['code_job'])[['code_job', 'job']].head()

### Lab 3 : 직업별 월급 차이 분석하기


1. 직업별 월급 평균표 만들기

In [ ]:
# 직업별 월급 평균표 만들기
job_income = welfare.dropna(subset=['job', 'income']) \
                    .groupby('job', as_index=False) \
                    .agg(mean_income = ('income', 'mean'))

In [ ]:
job_income.head()

2. 그래프 만들기

> (1) 월급이 많은 직업

In [ ]:
# 월급 상위 10위 추출
top10 = job_income.sort_values('mean_income', ascending=False).head(10)
top10

In [ ]:
# 그래프 그리기 / 한글 폰트 설정
import matplotlib.pyplot as plt
plt.rcParams.update({'font.family' : 'Malgun Gothic'})

In [ ]:
# 막대 그래프 그리기
sns.barplot(data=top10, y='job', x='mean_income')

> (2) 월급이 적은 직업

In [ ]:
# 하위 10개 직업 추출
bottom10 = job_income.sort_values('mean_income').head(10)

In [ ]:
bottom10

In [ ]:
# 막대 그래프 그리기 (상위 월급 직업군과 스케일을 맞추기 위해 x 범위를 0 ~ 800만원으로 조정
# sns.barplot(data=bottom10, y='job', x='mean_income')
sns.barplot(data=bottom10, y='job', x='mean_income').set(xlim=(0, 800))

> 데이터 분석
- '기타 돌봄∙보건 및 개인 생활 서비스 종사자'의 월급이 평균 73만원으로 가장 적고
- 월급이 가장 많은 '의료 진료 전문가' 평균 781만원과 비교하여 열배가 넘는 격차가 발생 	

## 성별 직업 빈도 - 성별로 어떤 직업이 가장 많을까?

### Lab 4 : 성별 직업 빈도 분석하기


1. 성별 직업 빈도표 만들기

In [ ]:
# 남성 직업 빈도 상위 10개 추출
job_male = welfare.dropna(subset=['job'])\
                .query('sex=="male"')\
                .groupby('job', as_index=False)\
                .agg(n=('job', 'count'))\
                .sort_values('n', ascending=False)\
                .head(10)

In [ ]:
job_male

In [ ]:
# 여성 직업 빈도 상위 10개 추출

job_female = welfare.dropna(subset=['job'])\
                .query('sex=="female"')\
                .groupby('job', as_index=False)\
                .agg(n=('job', 'count'))\
                .sort_values('n', ascending=False)\
                .head(10)

In [ ]:
job_female

2. 그래프 만들기

In [ ]:
# 남성 직업 빈도 막대 그래프 만들기
sns.barplot(data=job_male, x='n', y='job').set(xlim = (0, 500))

In [ ]:
# 여성 직업 빈도 막대 그래프 만들기
sns.barplot(data=job_female, x='n', y='job').set(xlim = (0, 500))

## 종교 유무에 따른 이혼율 - 종교가 있으면 이혼을 덜할까?
- 종교와 이혼을 관계 분석

## Lab 5 : 종교 변수 검토 및 전처리하기
- religion 변수 상태 검토

1. 변수 검토하기

In [ ]:
# 종교 변수 타입 출력
welfare['religion'].dtypes

In [ ]:
# 종교 별 빈도 구하기
welfare['religion'].value_counts()

> 종교 값이 코드화 되어 있음 (무응답(9)은 없음)

|   값   |  내용   |
|---     | ----    |
| 1      | 있음    |
| 2      | 없음    |
| 9      | 무응답  |

2. 전처리
- 종교가 있으면(1) 'yes', 없으면(2) 'no'로 변환

In [ ]:
# 종교 유무에 이름 부여
welfare['religion'] = np.where(welfare['religion'] == 1, 'yes', 'no')

In [ ]:
welfare['religion'].head()

In [ ]:
# 빈도 구하기
welfare['religion'].value_counts()

In [ ]:
# 막대 그래프 그리기
sns.countplot(data=welfare, x='religion')

## Lab 6 : 혼인상태 변수 검토 및 전처리하기
- marriage_type 변수 검토

1. 변수 검토하기

In [ ]:
# 변수 타입 출력
welfare['marriage_type'].dtypes

In [ ]:
# 결홍 유형에 따른 빈도수 출력
welfare['marriage_type'].value_counts()

> marriage_type이 0, 1, 2, 3, 4, 5, 6까지 7개 유형이 있음을 알 수 있다

|    값  |     내용     |
| ---    | -----        |
|0       | 해당 없음(18세 미만)    |
|1       | 유배우  |
|2       | 사별    |
|3       | 이혼    |
|4       | 별거    |
|5       | 미혼(18세 이상, 미혼모 포함    |
|6       | 기타(사망 등)    |

2. 파생변수 만들기 - 이혼 여부
- marriage_type (1) : 결혼
- marriage_type (3) : 이혼
- marriage_type (0, 2, 4, 5, 6) : 기타

In [ ]:
# 이혼 여부 변수 만들기
welfare['marriage'] = np.where(welfare['marriage_type'] == 1, 'marriage',
                      np.where(welfare['marriage_type'] == 3, 'divorce',
                                                          'etc'))

In [ ]:
# 이혼 여부별 빈도
n_divorce = welfare.groupby('marriage', as_index = False) \
                    .agg(n = ('marriage', 'count'))

In [ ]:
n_divorce

In [ ]:
# 막대 그래프 그리기
sns.barplot(data=n_divorce, x='marriage', y='n')

## Lab 7 : 종교 유무에 따른 이혼율 분석하기

1. 종교 유무에 따른 이혼율 표 만들기

In [ ]:
rel_div = welfare.query('marriage != "etc"') \
                .groupby(['religion', 'marriage'], as_index = False)\
                .agg(n=('religion', 'count'))
rel_div

In [ ]:
rel_div = welfare.query('marriage != "etc"') \
                .groupby('religion', as_index = False)['marriage']\
                .value_counts()
rel_div

In [ ]:
rel_div = welfare.query('marriage != "etc"') \
                .groupby('religion', as_index = False)['marriage']\
                .value_counts(normalize = True)
rel_div

2. 그래프 만들기

In [ ]:
rel_div = rel_div.query('marriage == "divorce"') \
                .assign(proportion = rel_div['proportion']*100) \
                .round(1)
rel_div

In [ ]:
sns.barplot(data=rel_div, x='religion', y='proportion')

> 종교가 있는 경우 이혼율 8.0%, 종교가 없는 경우 이혼율 9.5% (의미 잇는 차이 인가 ?)

## Lab 8 : 연령대 및 종교 유무에 따른 이혼율 분석하기

In [ ]:
welfare.query('marriage != "etc"') \
                .groupby('ageg', as_index=False) \
                ['marriage'] \
                .value_counts()

1. 연령대 별 이혼율 표 만들기

In [ ]:
age_div = welfare.query('marriage != "etc"') \
                .groupby('ageg', as_index=False) \
                ['marriage'] \
                .value_counts(normalize = True)
age_div

> 초년층은 사례가 부족하여 분석 작업에서 제외

2. 연령대별 이혼율 그래프 그리기

In [ ]:
age_div = age_div.query('ageg != "young" & marriage == "divorce"')\
                    .assign(proportion = age_div['proportion'] * 100)\
                    .round(1)
age_div

In [ ]:
# 막대 그래프 만들기
sns.barplot(data=age_div, x='ageg', y='proportion')

3. 연령대 및 종교 유무에 따른 이혼율표 만들기
- 분석에서 초년층 제외

In [ ]:
age_rel_div = welfare.query('marriage != "etc" & ageg != "young"') \
                    .groupby(['ageg', 'religion'], as_index=False) \
                    ['marriage'] \
                    .value_counts(normalize=True)
age_rel_div

4. 연령대 및 종교 유무에 따른 이혼율 그래프 만들기

In [ ]:
age_rel_div = \
    age_rel_div.query('marriage == "divorce"') \
                .assign(proportion = age_rel_div['proportion']*100) \
                .round(1)
age_rel_div

In [ ]:
# 막대 그래프 그리기
sns.barplot(data=age_rel_div, x='ageg', y='proportion', hue='religion')

##  지역별 연령대 비율 - 어느 지역에 노년층이 많을까?


## Lab 9 : 지역변수 검토 및 전처리

1. 변수 검토하기

In [ ]:
welfare['code_region'].dtypes

In [ ]:
welfare['code_region'].value_counts()

|    값  |     내용     |
| ---    | -----        |
|1       |서울   |
|2       | 수도권(인천, 경기)  |
|3       | 부산/경남/울산   |
|4       | 대구/경북    |
|5       | 대전/충남    |
|6       | 강원/충북    |
|7       | 광주/전남/전북/제주도    |

2. 전처리 하기

In [ ]:
# 지역 코드 목록 만들기
list_region = pd.DataFrame({'code_region' : [1, 2, 3, 4, 5, 6, 7],
                             'region' : ['서울',
                                         '수도권(인천,경기)',
                                         '부산/경남/울산',
                                          '대구/경북',
                                          '대전/충남',
                                           '강원/충북',
                                           '광주/전남/전북/제주도']})
list_region

In [ ]:
# 지역명 추가
welfare = welfare.merge(list_region, how='left', on='code_region')
welfare[['code_region', 'region']].head()

## Lab 10 : 지역별 연령대 비율 분석하기

1. 지역별 연령대 비율표 만들기

In [ ]:
region_ageg = welfare.groupby('region', as_index=False) \
                    ['ageg'] \
                    .value_counts(normalize=True)
region_ageg

2. 그래프 만들기

In [ ]:
region_ageg = \
  region_ageg.assign(proportion = region_ageg['proportion'] * 100) \
            .round(1)
region_ageg

In [ ]:
# 막대 그래프 만들기
sns.barplot(data=region_ageg, y='region', x='proportion', hue='ageg')

3. 누적 막대그래프 만들기

> (1) 피벗하기

In [ ]:
pivot_df = region_ageg[['region', 'ageg','proportion']].pivot(index = 'region',
                                                            columns = 'ageg',
                                                                values = 'proportion')
pivot_df

> (1) 그래프 만들기

In [ ]:
pivot_df.plot.barh(stacked = True)